# LLM invoice validation bench (Colab)

Просто запускай ячейки сверху вниз: установка зависимостей, загрузка payload, прогон моделей. Репозиторий ожидается в `/content/OCR`; если путь другой — поправь `repo_path` ниже.


In [ ]:
!pip install -q transformers accelerate bitsandbytes
# Для Llama 3.1 нужен HF токен:
# from huggingface_hub import login; login()

In [ ]:
import json, pathlib, sys
repo_path = pathlib.Path('/content/OCR')
if not repo_path.exists():
    raise RuntimeError('Смонтируй/распакуй репозиторий в /content/OCR или поправь repo_path')
sys.path.insert(0, str(repo_path))
from src.llm.runner import load_payload, run_model
from src.llm.models import MODEL_REGISTRY

payload_path = repo_path / 'src/llm/data/sample_payload.json'
payload = load_payload(payload_path)
out_dir = repo_path / 'src/llm/data/out'
out_dir.mkdir(parents=True, exist_ok=True)
model_keys = [
    'llama31-8b', 'llama31-70b', 'qwen2-7b', 'qwen2-72b',
    'mistral-7b', 'mixtral-8x7b', 'deepseek-7b', 'phi3-medium'
]
print('Will run models:', model_keys)


In [ ]:
results = {}
for key in model_keys:
    print(f'>>> Running {key}')
    res = run_model(key, payload, max_new_tokens=512, temperature=0.2)
    out_path = out_dir / f'{key}.json'
    out_path.write_text(json.dumps(res, indent=2, ensure_ascii=False), encoding='utf-8')
    results[key] = out_path.as_posix()

print('Saved outputs:')
print(json.dumps(results, indent=2))
